<img src="images/dask_horizontal.svg" align="right" width="30%">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high-level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Create data

In [1]:
%run prep.py -d accounts

## Setup

Again, we'll use the distributed scheduler. Schedulers will be explained in depth [later](05_distributed.ipynb).

In [2]:
from dask.distributed import Client

client = Client(n_workers=4)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [3]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [4]:
# each element is a text file, where each line is a JSON object
# note that the compression is handled automatically
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "Ingrid", "transactions": [{"transaction-id": 381, "amount": 950}, {"transaction-id": 471, "amount": 940}, {"transaction-id": 2194, "amount": 865}, {"transaction-id": 2306, "amount": 916}, {"transaction-id": 2361, "amount": 880}, {"transaction-id": 2863, "amount": 873}, {"transaction-id": 3356, "amount": 866}, {"transaction-id": 3711, "amount": 952}, {"transaction-id": 4372, "amount": 923}, {"transaction-id": 4375, "amount": 858}, {"transaction-id": 4650, "amount": 940}, {"transaction-id": 5597, "amount": 897}, {"transaction-id": 5697, "amount": 922}, {"transaction-id": 8245, "amount": 921}, {"transaction-id": 8332, "amount": 883}]}\n',)

In [5]:
# Edit sources.py to configure source locations
import sources
sources.bag_url

's3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv'

In [6]:
# Requires `s3fs` library
# each partition is a remote CSV text file
b = db.read_text(sources.bag_url,
                 storage_options={'anon': True})
b.take(1)

('VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n',)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [7]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<lambda, npartitions=10>

In [8]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entries for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [9]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Ingrid", "transactions": [{"transaction-id": 381, "amount": 950}, {"transaction-id": 471, "amount": 940}, {"transaction-id": 2194, "amount": 865}, {"transaction-id": 2306, "amount": 916}, {"transaction-id": 2361, "amount": 880}, {"transaction-id": 2863, "amount": 873}, {"transaction-id": 3356, "amount": 866}, {"transaction-id": 3711, "amount": 952}, {"transaction-id": 4372, "amount": 923}, {"transaction-id": 4375, "amount": 858}, {"transaction-id": 4650, "amount": 940}, {"transaction-id": 5597, "amount": 897}, {"transaction-id": 5697, "amount": 922}, {"transaction-id": 8245, "amount": 921}, {"transaction-id": 8332, "amount": 883}]}\n',
 '{"id": 1, "name": "Sarah", "transactions": [{"transaction-id": 201, "amount": 1619}, {"transaction-id": 625, "amount": 1138}, {"transaction-id": 1118, "amount": 1023}, {"transaction-id": 1180, "amount": 1100}, {"transaction-id": 1369, "amount": 1153}, {"transaction-id": 1551, "amount": 1331}, {"transaction-id": 1580, "amount": 1018

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [10]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Ingrid',
  'transactions': [{'transaction-id': 381, 'amount': 950},
   {'transaction-id': 471, 'amount': 940},
   {'transaction-id': 2194, 'amount': 865},
   {'transaction-id': 2306, 'amount': 916},
   {'transaction-id': 2361, 'amount': 880},
   {'transaction-id': 2863, 'amount': 873},
   {'transaction-id': 3356, 'amount': 866},
   {'transaction-id': 3711, 'amount': 952},
   {'transaction-id': 4372, 'amount': 923},
   {'transaction-id': 4375, 'amount': 858},
   {'transaction-id': 4650, 'amount': 940},
   {'transaction-id': 5597, 'amount': 897},
   {'transaction-id': 5697, 'amount': 922},
   {'transaction-id': 8245, 'amount': 921},
   {'transaction-id': 8332, 'amount': 883}]},
 {'id': 1,
  'name': 'Sarah',
  'transactions': [{'transaction-id': 201, 'amount': 1619},
   {'transaction-id': 625, 'amount': 1138},
   {'transaction-id': 1118, 'amount': 1023},
   {'transaction-id': 1180, 'amount': 1100},
   {'transaction-id': 1369, 'amount': 1153},
   {'transaction-id': 15

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [11]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 3,
  'name': 'Alice',
  'transactions': [{'transaction-id': 383, 'amount': 620},
   {'transaction-id': 1626, 'amount': 413},
   {'transaction-id': 1791, 'amount': 539},
   {'transaction-id': 2054, 'amount': 498},
   {'transaction-id': 2686, 'amount': 598},
   {'transaction-id': 2938, 'amount': 542},
   {'transaction-id': 3339, 'amount': 572},
   {'transaction-id': 4668, 'amount': 581},
   {'transaction-id': 6776, 'amount': 495},
   {'transaction-id': 7182, 'amount': 516},
   {'transaction-id': 7588, 'amount': 648},
   {'transaction-id': 8900, 'amount': 534},
   {'transaction-id': 9340, 'amount': 557}]},
 {'id': 26,
  'name': 'Alice',
  'transactions': [{'transaction-id': 133, 'amount': 6},
   {'transaction-id': 483, 'amount': 6},
   {'transaction-id': 640, 'amount': 4},
   {'transaction-id': 1399, 'amount': 5},
   {'transaction-id': 1966, 'amount': 7},
   {'transaction-id': 2511, 'amount': 5},
   {'transaction-id': 2935, 'amount': 7},
   {'transaction-id': 3514, 'amount': 9},
 

In [12]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 13},
 {'name': 'Alice', 'count': 24},
 {'name': 'Alice', 'count': 5},
 {'name': 'Alice', 'count': 4},
 {'name': 'Alice', 'count': 124})

In [13]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(13, 24, 5, 4, 124)

In [14]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

25.304621848739497

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [15]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .take(3))

([{'transaction-id': 383, 'amount': 620},
  {'transaction-id': 1626, 'amount': 413},
  {'transaction-id': 1791, 'amount': 539},
  {'transaction-id': 2054, 'amount': 498},
  {'transaction-id': 2686, 'amount': 598},
  {'transaction-id': 2938, 'amount': 542},
  {'transaction-id': 3339, 'amount': 572},
  {'transaction-id': 4668, 'amount': 581},
  {'transaction-id': 6776, 'amount': 495},
  {'transaction-id': 7182, 'amount': 516},
  {'transaction-id': 7588, 'amount': 648},
  {'transaction-id': 8900, 'amount': 534},
  {'transaction-id': 9340, 'amount': 557}],
 [{'transaction-id': 133, 'amount': 6},
  {'transaction-id': 483, 'amount': 6},
  {'transaction-id': 640, 'amount': 4},
  {'transaction-id': 1399, 'amount': 5},
  {'transaction-id': 1966, 'amount': 7},
  {'transaction-id': 2511, 'amount': 5},
  {'transaction-id': 2935, 'amount': 7},
  {'transaction-id': 3514, 'amount': 9},
  {'transaction-id': 3544, 'amount': 7},
  {'transaction-id': 4245, 'amount': 6},
  {'transaction-id': 4354, 'amount

In [16]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 383, 'amount': 620},
 {'transaction-id': 1626, 'amount': 413},
 {'transaction-id': 1791, 'amount': 539})

In [17]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(620, 413, 539)

In [18]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

858.082191780822

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [19]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [20]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [21]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [22]:
b.foldby(lambda x: x % 2, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [23]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

[('Alice', 112), ('Alice', 114), ('Alice', 115), ('Alice', 135), ('Bob', 119), ('Bob', 119), ('Bob', 120), ('Bob', 140), ('Charlie', 139), ('Charlie', 141), ('Charlie', 142), ('Charlie', 163), ('Dan', 60), ('Dan', 60), ('Dan', 60), ('Dan', 70), ('Edith', 116), ('Edith', 117), ('Edith', 117), ('Edith', 137), ('Frank', 36), ('Frank', 36), ('Frank', 36), ('Frank', 42), ('George', 117), ('George', 120), ('George', 120), ('George', 138), ('Hannah', 79), ('Hannah', 82), ('Hannah', 83), ('Hannah', 96), ('Ingrid', 154), ('Ingrid', 155), ('Ingrid', 156), ('Ingrid', 181), ('Jerry', 92), ('Jerry', 92), ('Jerry', 93), ('Jerry', 111), ('Kevin', 84), ('Kevin', 84), ('Kevin', 84), ('Kevin', 98), ('Laura', 139), ('Laura', 139), ('Laura', 140), ('Laura', 161), ('Michael', 111), ('Michael', 112), ('Michael', 112), ('Michael', 133), ('Norbert', 168), ('Norbert', 168), ('Norbert', 168), ('Norbert', 196), ('Oliver', 87), ('Oliver', 88), ('Oliver', 92), ('Oliver', 99), ('Patricia', 118), ('Patricia', 118), 

In [24]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot + 1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 476), ('Bob', 498), ('Charlie', 585), ('Dan', 250), ('Edith', 487), ('Frank', 150), ('George', 495), ('Hannah', 340), ('Ingrid', 646), ('Jerry', 388), ('Kevin', 350), ('Laura', 579), ('Michael', 468), ('Norbert', 700), ('Oliver', 366), ('Patricia', 492), ('Quinn', 427), ('Ray', 312), ('Sarah', 805), ('Tim', 400), ('Ursula', 295), ('Victor', 750), ('Wendy', 495), ('Xavier', 538), ('Yvonne', 500), ('Zelda', 349)]
CPU times: user 102 ms, sys: 5.45 ms, total: 107 ms
Wall time: 379 ms


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [25]:
# Your code here...

## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from the point of view of a Bag, it is frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [26]:
df1 = js.to_dataframe()
df1.head()

/usr/share/miniconda3/envs/dask-tutorial/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/usr/share/miniconda3/envs/dask-tutorial/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/usr/share/miniconda3/envs/dask-tutorial/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


,id,name,transactions
0,0,Ingrid,"[{'transaction-id': 381, 'amount': 950}, {'tra..."
1,1,Sarah,"[{'transaction-id': 201, 'amount': 1619}, {'tr..."
2,2,Victor,"[{'transaction-id': 49, 'amount': 1295}, {'tra..."
3,3,Alice,"[{'transaction-id': 383, 'amount': 620}, {'tra..."
4,4,Charlie,"[{'transaction-id': 216, 'amount': 4603}, {'tr..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` by more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [27]:
%time df1.groupby('name').id.count().compute().head()

CPU times: user 183 ms, sys: 8.26 ms, total: 191 ms
Wall time: 1.13 s


name
Alice      476
Bob        498
Charlie    585
Dan        250
Edith      487
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [28]:
def denormalize(record):
    # returns a list for each person, one item per transaction
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'Ingrid', 'amount': 950, 'transaction-id': 381},
 {'id': 0, 'name': 'Ingrid', 'amount': 940, 'transaction-id': 471},
 {'id': 0, 'name': 'Ingrid', 'amount': 865, 'transaction-id': 2194})

In [29]:
df = transactions.to_dataframe()
df.head()

,id,name,amount,transaction-id
0,0,Ingrid,950,381
1,0,Ingrid,940,471
2,0,Ingrid,865,2194
3,0,Ingrid,916,2306
4,0,Ingrid,880,2361


In [30]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

CPU times: user 143 ms, sys: 9.14 ms, total: 152 ms
Wall time: 864 ms


name
Alice       12045
Bob         16331
Charlie     17014
Dan         14266
Edith       40049
Frank        3196
George      21248
Hannah       5762
Ingrid      22674
Jerry       16726
Kevin       16654
Laura       16890
Michael     10128
Norbert     40832
Oliver       8800
Patricia    12730
Quinn       12704
Ray         24044
Sarah       30366
Tim         27746
Ursula       6587
Victor      44581
Wendy       11029
Xavier      24432
Yvonne      25396
Zelda       17770
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.

## Learn More

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Shutdown

In [31]:
client.shutdown()

/usr/share/miniconda3/envs/dask-tutorial/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/usr/share/miniconda3/envs/dask-tutorial/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/usr/share/miniconda3/envs/dask-tutorial/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/usr/share/mi